In [2]:
from bs4 import BeautifulSoup
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/75.0.3770.80 Safari/537.36"
}

## Thực hành một số trang web khác
Cách phân tích data từ 1 trang web dựa vào inspect và HTML tag<br>
Tìm đối tượng muốn trích xuất thông tin<br>
Tìm `href`, `class`, ...

In [56]:
# session = requests.Session()
# session.trust_env = True
proxies = {
  "http": None,
  "https": None,
}
main_url = "https://www.forexfactory.com/"
res = requests.get(main_url, headers=headers, proxies=proxies)
res

<Response [503]>

In [50]:
res

<Response [503]>

In [5]:
soup = BeautifulSoup(res.content, "html.parser")

Xem mội dung của từng bài trong trang chủ.

Trước hết cần tìm ID/CLASS của các item chứa nội dung bài viết

In [6]:
news = soup.findAll("div", {"class": "single_post_text"})

In [7]:
len(news)

95

Kiểm tra nội dung cần lấy trong mỗi item.

Mỗi item chỉ có thông tin về Title và đường dẫn đến link khác.

In [8]:
news[10]

<div class="single_post_text">
<p class="meta meta_style4">
<a class="active" href="/kinh-te/vi-mo.htm" title="VĨ MÔ">VĨ MÔ</a>
<span class="time-mobile"><i class="time-sub"> |  </i> 05/08 10:10</span>
</p>
<h4><a class="fontbold textshadow" href="/2022/08/bai-hoc-chong-lam-phat-761-989445.htm" title="Bài học chống lạm phát">Bài học chống lạm phát</a></h4>
</div>

Tiếp tục truy xuất attributes của item tag tùy thuộc vào nội dung muốn lấy (đường dẫn href)

In [9]:
href_tmp = news[12].h4.a.attrs["href"]
href_tmp

'/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm'

Ghép url lại với nhau, có thể thủ công hoặc library

In [10]:
import urllib.parse

Library

In [11]:
sub_url = urllib.parse.urljoin(main_url, href_tmp)
sub_url

'https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm'

Thủ công

In [18]:
if href_tmp.startswith("/") and main_url.endswith("/"):
    sub_url = main_url + href_tmp[1:]
sub_url

'https://vietstock.vn/2022/08/co-nen-san-nhan-tai-nhu-the-4265-987274.htm'

Lấy content của page item dựa vào url tương tự

In [12]:
sub_page_res = requests.get(sub_url, headers=headers)

In [13]:
sub_page_res

<Response [200]>

In [14]:
sub_soup = BeautifulSoup(sub_page_res.content, "html.parser")

Lấy thông tin head line

In [15]:
sub_soup_head = sub_soup.find("p", {"class": "pHead"})

In [17]:
sub_soup_head.text

'Còn nhớ việc báo giới từng đưa tin cổ phiếu HAG của Công ty Hoàng Anh Gia Lai đứng trước nguy cơ bị hủy niêm yết khỏi sàn giao dịch chứng khoán vào đầu năm 2022 do thua lỗ ba năm liên tiếp (2017, 2018 và 2019) không những là một cú sốc lớn đối với nhà đầu tư, cổ đông sở hữu cổ phiếu của Hoàng Anh Gia Lai mà còn ảnh hưởng tiêu cực đến thị trường chứng khoán Việt Nam tại thời điểm đó…'

Lấy thông tin về keywords của bài, để xem nội dung về những keywords nào

In [23]:
keywords = sub_soup.find("div", {"class": "tags mt20"})
keywords

<div class="tags mt20">
<ul class="inline">
<li class="tag_list"><i class="fas fa-tag"></i></li>
<li>
<a href="/tag/Hủy-niêm-yết-cổ-phiếu">Hủy niêm yết cổ phiếu</a>
</li>
<li>
<a href="/tag/-Những-cổ-phiếu-đưa-vào-diện-hủy-niêm-yết"> Những cổ phiếu đưa vào diện hủy niêm yết</a>
</li>
<li>
<a href="/tag/-Quy-định-về-hủy-niêm-yết"> Quy định về hủy niêm yết</a>
</li>
</ul>
</div>

Lấy nội dung của từng keywords

In [24]:
keywords = keywords.findAll("a")
keywords

[<a href="/tag/Hủy-niêm-yết-cổ-phiếu">Hủy niêm yết cổ phiếu</a>,
 <a href="/tag/-Những-cổ-phiếu-đưa-vào-diện-hủy-niêm-yết"> Những cổ phiếu đưa vào diện hủy niêm yết</a>,
 <a href="/tag/-Quy-định-về-hủy-niêm-yết"> Quy định về hủy niêm yết</a>]

In [21]:
keywords[0].text

'Hủy niêm yết cổ phiếu'

## Crawl bằng Selenium
Install `selenium` bằng `pip install selenium`<br>
Download webdriver (Chrome) ***nhưng phải cùng version với browser đang dùng***

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
# thư viện thay cho input
import getpass

In [44]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [35]:
driver.quit()

Tìm items muốn get bằng XPATH<br>
`Xpath=//tagname[@attribute='value']`<br>
- // : Select current node.
- Tagname: Tagname of the particular node.
- @: Select attribute.
- Attribute: Attribute name of the node.
- Value: Value of the attribute.

Hướng dẫn cách tìm và lấy XPath bằng browser + extension Chrome

#### 1) Basic XPath:
Absolute XPath<br>
`//html/body[@class='theme-4']/div[@class='padding-top-100 bg-event lr-banner-pos']/div[@class='bg-full']/div[@class='bg4']/div[@class='inernational4 mb30']/div[@class='container']/div[@class='row']/div[@class='col-md-12 col-xl-8']/div[@id='TieuDiem2']/div[@class='science_carousel owl-carousel nav_style4 owl-loaded owl-drag']/div[@class='owl-stage-outer']/div[@class='owl-stage']/div[@class='owl-item active'][1]/div[@class='science_carousel_items']/div[@class='single_post mb30 type18']/div[@class='single_post_text']`

--> Dễ bị failed khi có 1 thông tin trong path bị thay đổi. Không recommend dùng absolute XPath

Relative XPath<br>
`//div[@class='single_post_text']`<br>
Tương tự như dùng get element ở phương pháp trước<br>
--> Recommend dùng cách này, vì nếu có thay đổi thông tin nhiều elements trong path thì vẫn lấy được nội dung, trừ khi thay đổi thông tin của chính tag cần lấy :D

In [37]:
driver.get("https://vietstock.vn/")

In [38]:
pages = driver.find_elements(By.XPATH, "//div[@class='owl-stage']//div[@class='single_post_text']/h4/a")

In [29]:
len(pages)

36

In [31]:
dir(pages[0])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

#### Working around with a page

Truy xuất tag `a` trong element vừa lấy

In [32]:
x = pages[0].find_element(By.XPATH, "//h4/a")

Lấy thông tin `href` của thẻ

In [33]:
x.get_attribute("href")

'https://vietstock.vn/2022/08/bong-den-no-xau-ngan-hang-dang-hien-ro-757-990648.htm'

Redirect đến page của trang tương ứng, bằng hành động click vào element `a` vừa lấy ở trên

In [34]:
driver.execute_script("arguments[0].click();", x)

Hoặc mở trang web bằng một windows khác

In [ ]:
driver.execute_script("window.open('" + x.get_attribute("href") +"');")

Lưu ý khi mở url bằng windows khác thì cần set current windows cho browser, và để ý số lượng windows đang mở cũng như thứ tự mở chúng để dễ dàng thao tác với từng trang<br>
Tuy nhiên recommend close windows sau khi dùng xong để tránh nhầm lẫn<br>
- `driver.current_window_handle`
- `driver.window_handles`
- `current_wd = driver.window_handles[1]`
- `driver.switch_to.window(current_wd)`

Lấy thông tin về headline của trang web tương ứng<br>
Sử dụng contains thay vì exact match, có thể có variant trong việc đặt tên class

In [130]:
y = driver.find_element(By.XPATH, "//p[contains(@class, 'pHead')]")

Ngoài ra có thể sử dụng AND hay OR trong XPath

In [106]:
y = driver.find_element(By.XPATH, "//p[contains(@class, 'pHead') or @class='pHead xh-highlight']")

Check các page đã lấy được từ trang chủ

In [9]:
for page in pages:
    print(page.get_attribute("href"))

https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm
https://vietstock.vn/2022/08/ha-tang-giao-thong-sao-khong-tinh-lam-1-lan-cho-xong-4221-987961.htm
https://vietstock.vn/2022/08/co-nen-san-nhan-tai-nhu-the-4265-987274.htm
https://vietstock.vn/2022/08/bai-hoc-chong-lam-phat-761-989445.htm
https://vietstock.vn/2022/08/co-che-dac-thu-cho-tphcm-chung-ta-co-the-mong-doi-nhieu-hon-the-761-989072.htm
https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm
https://vietstock.vn/2022/08/ha-tang-giao-thong-sao-khong-tinh-lam-1-lan-cho-xong-4221-987961.htm
https://vietstock.vn/2022/08/co-nen-san-nhan-tai-nhu-the-4265-987274.htm
https://vietstock.vn/2022/08/bai-hoc-chong-lam-phat-761-989445.htm
https://vietstock.vn/2022/08/co-che-dac-thu-cho-tphcm-chung-ta-co-the-mong-doi-nhieu-hon-the-761-989072.htm
https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm
https://vietstock.vn/2022/08/co-phieu-nao-thuong-tang-vao-than

In [39]:
import time
headlines = []
main_window = driver.current_window_handle
for i, page in enumerate(pages[:5]):
    # try:
    url = page.get_attribute("href")
    driver.execute_script("window.open('" + url +"');")
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(4)
    try:
        sub_page = driver.find_element(By.XPATH, "//p[contains(@class, 'pHead')]")
    except:
        driver.close()
        driver.switch_to.window(main_window)
    headline = sub_page.text
    headlines.append((url, headline))
    driver.close()
    driver.switch_to.window(main_window)
    time.sleep(4)
    print(headlines[-1][0])
    # except:
    #     pass

https://vietstock.vn/2022/08/bai-hoc-chong-lam-phat-761-989445.htm
https://vietstock.vn/2022/08/co-che-dac-thu-cho-tphcm-chung-ta-co-the-mong-doi-nhieu-hon-the-761-989072.htm
https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm
https://vietstock.vn/2022/08/trong-cung-hay-trong-cau-118-990613.htm
https://vietstock.vn/2022/08/thue-thu-nhap-ca-nhan-qua-loi-thoi-so-voi-10-nam-truoc-758-989895.htm


In [40]:
headlines

[('https://vietstock.vn/2022/08/bai-hoc-chong-lam-phat-761-989445.htm',
  'Trong một động thái hiếm thấy, nhà kinh tế đoạt giải Nobel Paul Krugman vừa viết một bài đăng trên tờ New York Times với một tít thừa nhận: “Tôi đã sai về lạm phát”.'),
 ('https://vietstock.vn/2022/08/co-che-dac-thu-cho-tphcm-chung-ta-co-the-mong-doi-nhieu-hon-the-761-989072.htm',
  'TPHCM đang tiến hành các hội nghị lấy ý kiến đóng góp cho Dự thảo báo cáo tổng kết 10 năm thực hiện Nghị quyết 16 của Bộ Chính trị về phương hướng, nhiệm vụ phát triển TPHCM đến năm 2020 và 5 năm thực hiện Nghị quyết 54 của Quốc hội về thí điểm cơ chế, chính sách đặc thù phát triển TPHCM.'),
 ('https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm',
  'Còn nhớ việc báo giới từng đưa tin cổ phiếu HAG của Công ty Hoàng Anh Gia Lai đứng trước nguy cơ bị hủy niêm yết khỏi sàn giao dịch chứng khoán vào đầu năm 2022 do thua lỗ ba năm liên tiếp (2017, 2018 và 2019) không những là một cú sốc lớn đối với nhà đầu t

In [26]:
driver.execute_script("window.open('" + pages[0].get_attribute("href") +"');")

In [48]:
driver.switch_to.window(main_window)

In [13]:
driver.window_handles

['CDwindow-AA6387CDB193BA173CB941AC7144FE22']

In [44]:
driver.close()

In [38]:
driver.current_window_handle

'CDwindow-B93336B735AF5F6700F9B62B97C34F4B'

In [14]:
headlines

[('https://vietstock.vn/2022/08/huy-niem-yet-co-phieu-can-quy-dinh-ro-830-988671.htm',
  'Còn nhớ việc báo giới từng đưa tin cổ phiếu HAG của Công ty Hoàng Anh Gia Lai đứng trước nguy cơ bị hủy niêm yết khỏi sàn giao dịch chứng khoán vào đầu năm 2022 do thua lỗ ba năm liên tiếp (2017, 2018 và 2019) không những là một cú sốc lớn đối với nhà đầu tư, cổ đông sở hữu cổ phiếu của Hoàng Anh Gia Lai mà còn ảnh hưởng tiêu cực đến thị trường chứng khoán Việt Nam tại thời điểm đó…'),
 ('https://vietstock.vn/2022/08/ha-tang-giao-thong-sao-khong-tinh-lam-1-lan-cho-xong-4221-987961.htm',
  'Mới đây Sở Giao thông-Vận tải (GT-VT) TPHCM đề xuất mở rộng cao tốc TPHCM - Trung Lương từ 6 làn xe lên 8 làn xe, với lý do cao tốc này hiện đã quá tải, thường xuyên ùn tắc, mỗi ngày có hơn 51.000 lượt xe qua lại, nếu không mở rộng sẽ gây tác động xấu đến thực trạng GTVT, đặc biệt là vào các dịp lễ tết.'),
 ('https://vietstock.vn/2022/08/co-nen-san-nhan-tai-nhu-the-4265-987274.htm',
  'Trước thông tin TPHCM thu 

In [41]:
driver.quit()

#### Cung cấp username/password nếu cần thiết

In [42]:
username = getpass.getpass("Enter username: ")
password = getpass.getpass("Enter password: ")

In [45]:
driver.get("https://trade.vndirect.com.vn/chung-khoan/danh-muc")
user_name = driver.find_element("xpath", "//div[@class='form-field']/input[@name='username']")
user_name.send_keys(username)
pass_word = driver.find_element("xpath", "//div[@class='form-field']/input[@name='password']")
pass_word.send_keys(password)

In [46]:
button = driver.find_element(By.XPATH, "//div[@id='login-popup']/form/button")
button.click()

Ngoài ra còn có nhiều tính năng khá hay của selenium

Scroll trong trang web

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [47]:
driver.quit()

In [48]:
button2 = driver.find_element(By.XPATH, "//*[@id='ui-inner']/header[1]/div/div[1]/ul[1]/li[3]")

MaxRetryError: HTTPConnectionPool(host='localhost', port=59168): Max retries exceeded with url: /session/bafcd21533829d886ba1e5a26edeb12a/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x138878040>: Failed to establish a new connection: [Errno 61] Connection refused'))